In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=37, out_features=2048)
        self.layer_2 = nn.Linear(in_features=2048, out_features=1024) 
        self.layer_3 = nn.Linear(in_features=1024, out_features=512)
        self.layer_4 = nn.Linear(in_features=512, out_features=128)
        self.layer_5 = nn.Linear(in_features=128, out_features=1)
    
    def forward(self, x):
        out = torch.relu(self.layer_1(x))
        out = torch.relu(self.layer_2(out))
        out = torch.relu(self.layer_3(out))
        out = torch.relu(self.layer_4(out))
        out = torch.sigmoid(self.layer_5(out))
        return out

loaded_model = Net().to("cuda")
loaded_model.load_state_dict(torch.load('model2.pt',map_location='cuda'))
loaded_model

Net(
  (layer_1): Linear(in_features=37, out_features=2048, bias=True)
  (layer_2): Linear(in_features=2048, out_features=1024, bias=True)
  (layer_3): Linear(in_features=1024, out_features=512, bias=True)
  (layer_4): Linear(in_features=512, out_features=128, bias=True)
  (layer_5): Linear(in_features=128, out_features=1, bias=True)
)

In [50]:
# pre-process the data: converting categorical data to numerical data
data_path = "./data_test.csv"
data_pd = pd.read_csv(data_path)
categorical_data = data_pd.select_dtypes(include=["object"])
categorical_data_onehot = pd.get_dummies(categorical_data).to_numpy()
numerical_data = data_pd.select_dtypes(exclude=['object']).to_numpy()
numerical_data = torch.from_numpy(numerical_data).to(torch.float32)

# normalization
numerical_data = numerical_data[:,:14]
mean = numerical_data.mean(dim=0)
std = numerical_data.std(dim=0)
numerical_data = (numerical_data - mean) / std

# merge the training data
merged_data = np.concatenate((numerical_data, categorical_data_onehot), axis=1)
merged_data, merged_data.shape,numerical_data.shape,categorical_data_onehot.shape

(array([[-0.91449547, -0.2730752 , -0.9502393 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.21173128, -1.0393502 ,  0.02875139, ...,  0.        ,
          0.        ,  0.        ],
        [-0.7893592 ,  0.49319977, -0.8278655 , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [-2.290995  , -1.8056251 , -2.785847  , ...,  0.        ,
          0.        ,  0.        ],
        [-1.0396318 , -1.0393502 ,  0.02875139, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.4620039 , -1.0393502 ,  0.3958729 , ...,  0.        ,
          0.        ,  0.        ]], dtype=float32),
 (2026, 37),
 torch.Size([2026, 14]),
 (2026, 23))

In [51]:
data = torch.from_numpy(merged_data).to(torch.float32).to('cuda')    # to tensor
data

tensor([[-0.9145, -0.2731, -0.9502,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2117, -1.0394,  0.0288,  ...,  0.0000,  0.0000,  0.0000],
        [-0.7894,  0.4932, -0.8279,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-2.2910, -1.8056, -2.7858,  ...,  0.0000,  0.0000,  0.0000],
        [-1.0396, -1.0394,  0.0288,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.4620, -1.0394,  0.3959,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')

In [57]:
loaded_model.eval()

with torch.no_grad():  # Disable gradient tracking for inference
    predictions = loaded_model(data)
    
threshold = 0.8
predictions = (predictions >= threshold).int()

predictions = predictions.tolist()

In [59]:
import csv

# Assuming 'predictions' is a list or array containing your model's predictions

# Specify the file name for the output CSV file
output_file = "test_output_model2.csv"

# Open the file in write mode with newline='' to avoid extra line breaks
with open(output_file, "w", newline="") as csvfile:
    # Create a CSV writer object
    csv_writer = csv.writer(csvfile)

    # Write each prediction as a separate row without headers
    for prediction in predictions:
        csv_writer.writerow(prediction)




